In [2]:
# import autograd ’s automatic differentiator
from autograd import grad
from autograd import hessian

# import NumPy library
import numpy as np

# Newton ’s method
def newtons_method (g, max_its, w):

    # compute gradient/ Hessian using autograd
    gradient = grad(g)
    hess = hessian(g)
    
    # set numerical stability parameter
    epsilon = 10 **(-7)
    if 'epsilon' in kwargs:
        epsilon = kwargs['epsilon']
    
    # run the Newton ’s method loop
    weight_history = [w] # container for weight history
    cost_history = [g(w)] # container for cost function history
    for k in range( max_its ):
    
        # evaluate the gradient and hessian
        grad_eval = gradient (w)
        hess_eval = hess(w)
    
        # reshape hessian to square matrix
        hess_eval .shape = (int((np. size( hess_eval ))**(0.5)),int((np.
                            size( hess_eval ))**(0.5)))
    
        # solve second -order system for weight update
        A = hess_eval + epsilon* np. eye(w. size)
        b = grad_eval
        w = np. linalg. solve(A, np .dot(A ,w)-b)
    
        # record weight and cost
        weight_history. append(w)
        cost_history.append(g(w))
    
    return weight_history, cost_history

In [7]:
import autograd.numpy.random as npr

def x2(x):
    return x*x

g = grad(x2(2))

print (g(npr.randn(10)))

TypeError: 'int' object is not callable

In [8]:
"""This example shows how to define the gradient of your own functions.
This can be useful for speed, numerical stability, or in cases where
your code depends on external library calls."""
from __future__ import absolute_import
from __future__ import print_function
import autograd.numpy as np
import autograd.numpy.random as npr

from autograd import grad
from autograd.extend import primitive, defvjp
from autograd.test_util import check_grads


# @primitive tells Autograd not to look inside this function, but instead
# to treat it as a black box, whose gradient might be specified later.
# Functions with this decorator can contain anything that Python knows
# how to execute, and you can do things like in-place operations on arrays.
@primitive
def logsumexp(x):
    """Numerically stable log(sum(exp(x))), also defined in scipy.special"""
    max_x = np.max(x)
    return max_x + np.log(np.sum(np.exp(x - max_x)))

# Next, we write a function that specifies the gradient with a closure.
# The reason for the closure is so that the gradient can depend
# on both the input to the original function (x), and the output of the
# original function (ans).

def logsumexp_vjp(ans, x):
    # If you want to be able to take higher-order derivatives, then all the
    # code inside this function must be itself differentiable by Autograd.
    # This closure multiplies g with the Jacobian of logsumexp (d_ans/d_x).
    # Because Autograd uses reverse-mode differentiation, g contains
    # the gradient of the objective w.r.t. ans, the output of logsumexp.
    # This returned VJP function doesn't close over `x`, so Python can
    # garbage-collect `x` if there are no references to it elsewhere.
    x_shape = x.shape
    return lambda g: np.full(x_shape, g) * np.exp(x - np.full(x_shape, ans))

# Now we tell Autograd that logsumexmp has a gradient-making function.
defvjp(logsumexp, logsumexp_vjp)

if __name__ == '__main__':
    # Now we can use logsumexp() inside a larger function that we want
    # to differentiate.
    def example_func(y):
        z = y**2
        lse = logsumexp(z)
        return np.sum(lse)

    grad_of_example = grad(example_func)
    print("Gradient: \n", grad_of_example(npr.randn(10)))

    # Check the gradients numerically, just to be safe.
    check_grads(example_func, modes=['rev'])(npr.randn(10))

Gradient: 
 [ 1.16136957e-03 -3.58870504e-01  4.81462918e+00  1.08097338e-02
  3.33528185e-04 -1.06304081e-01 -3.06515432e-04 -4.67518265e-05
 -2.98901620e-02  4.22077985e-03]
